In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


2.3.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
torch.cuda.is_available()

False

the diffrent steps of GCNConv is


*   Add self-loops to the adjacency matrix.
*   Linearly transform node feature matrix.
*   Compute normalization coefficients.
*   Normalize node features
*   Sum up neighboring node features ("add" aggregation).
*   Apply a final bias vector.







In [ ]:
import torch
from torch.nn import Linear, Parameter
from torch_geometric.utils import degree,add_self_loops
class GCNConv(torch.nn.Module):
  def __init__(self, in_channal, out_chennal) -> None:
    super().__init__(aggr='add')
    self.lin=Linear(in_channal=in_channal, out_chennal=out_chennal)
    self.bais=Parameter(torch.empty(out_chennal))
    self.reset_perematers()

  def reset_perematers(self):
    self.lin.reset_perematers()
    self.bais.data.zero
  def forward(self, x, edges_endex):

        edges_endex, _ =add_self_loops(edge_index=edges_endex,num_nodes=x.size(0))

        row,col=edges_endex
        deg=degree(col, x.size(0))
        deg_inv_sqrt=deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        out = self.propagate(edges_endex, x=x, norm=norm)
        out=out+self.bais
        return out

  def message(self, x_j, norm):

        return norm.view(-1, 1) * x_j


In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]
data


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

create the model

In [ ]:

import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.empty(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):

        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))


        x = self.lin(x)


        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]


        out = self.propagate(edge_index, x=x, norm=norm)


        out = out + self.bias

        return out

    def message(self, x_j, norm):

        return norm.view(-1, 1) * x_j

In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv()
  (conv2): GCNConv()
)


In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc


for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Loss: 1.9460
Epoch: 002, Loss: 1.9437
Epoch: 003, Loss: 1.9405
Epoch: 004, Loss: 1.9371
Epoch: 005, Loss: 1.9323
Epoch: 006, Loss: 1.9272
Epoch: 007, Loss: 1.9240
Epoch: 008, Loss: 1.9186
Epoch: 009, Loss: 1.9130
Epoch: 010, Loss: 1.9054
Epoch: 011, Loss: 1.9016
Epoch: 012, Loss: 1.8953
Epoch: 013, Loss: 1.8841
Epoch: 014, Loss: 1.8809
Epoch: 015, Loss: 1.8704
Epoch: 016, Loss: 1.8653
Epoch: 017, Loss: 1.8556
Epoch: 018, Loss: 1.8441
Epoch: 019, Loss: 1.8357
Epoch: 020, Loss: 1.8263
Epoch: 021, Loss: 1.8163
Epoch: 022, Loss: 1.8092
Epoch: 023, Loss: 1.8015
Epoch: 024, Loss: 1.7846
Epoch: 025, Loss: 1.7745
Epoch: 026, Loss: 1.7562
Epoch: 027, Loss: 1.7480
Epoch: 028, Loss: 1.7301
Epoch: 029, Loss: 1.7136
Epoch: 030, Loss: 1.7053
Epoch: 031, Loss: 1.6922
Epoch: 032, Loss: 1.6709
Epoch: 033, Loss: 1.6560
Epoch: 034, Loss: 1.6509
Epoch: 035, Loss: 1.6360
Epoch: 036, Loss: 1.5798
Epoch: 037, Loss: 1.5886
Epoch: 038, Loss: 1.5670
Epoch: 039, Loss: 1.5472
Epoch: 040, Loss: 1.5170


In [ ]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.8110
